In [ ]:
from bs4 import BeautifulSoup as bts

import requests
from urllib.request import urlopen

import pandas as pd

import re

import time
import sys
from datetime import date, datetime

In [ ]:
def getAndParseURL(url):
    result = requests.get(url, headers={"User-Agent":"Mozilla/5.0"})
    soup = bts(result.text, 'html.parser')
    return soup

In [ ]:
url = "https://www.imdb.com/chart/top"
soup = getAndParseURL(url)
movie_urls = []
for i in soup.tbody.findAll('tr'):
    movie_urls.append('https://www.imdb.com'+i.findAll('td')[1].a['href'])

In [ ]:
row = []
count = 0
for i in movie_urls:
    
    count += 1
    html = getAndParseURL(i)

    try:
        title = html.find('h1', {"data-testid":"hero-title-block__title"}).text
    except:
        title = None
        print("Couldn't scrape the title data")

    try:
#         revenue = html.find('li', {"data-testid":"title-boxoffice-cumulativeworldwidegross"}).findAll('span')[1].text
        revenue = int(re.findall("\d+", ''.join(html.find('li', {"data-testid":"title-boxoffice-cumulativeworldwidegross"}).findAll('span')[1].text.split(',')))[0])
    except:
        revenue = None
        print("Couldn't scrape the revenue data")

    try:
#         budget = html.find('li', {"data-testid":"title-boxoffice-budget"}).findAll('span')[1].text.split(' ')[0]
        budget = int(re.findall("\d+", ''.join(html.find('li', {"data-testid":"title-boxoffice-budget"}).li.text.split(',')))[0])
    except:
        budget = None
        print("Couldn't scrape the budget data")
        
    try:
        runtime = html.find('ul', {"data-testid":"hero-title-block__metadata"}).findAll('li')[2].text
    except:
        runtime = None
        print("Couldn't scrape the runtime data")
        
    try:
        certificate = html.find('ul', {"data-testid":"hero-title-block__metadata"}).findAll('li')[1].span.text
    except:
        certificate = None
        print("Couldn't scrape the certificate data")

    try:
        releaseDate = pd.to_datetime(html.find('li', {"data-testid":"title-details-releasedate"}).li.a.text.split('(')[0].strip())
    except:
        releaseDate = None
        print("Couldn't scrape the date data")

    try:
        score = int(10*float(html.find('div', {"data-testid":"hero-rating-bar__aggregate-rating__score"}).text.split('/')[0]))
    except:
        score = None
        print("Couldn't scrape the score data")

    try:
        votes = html.find('div', {"data-testid":"hero-rating-bar__aggregate-rating"}).a.div.div.findAll('div')[1].findAll('div')[2].text
        if 'M' in votes:
            votes = int(float(votes.split('M')[0])*1_000_000)
        elif 'K' in votes:
            votes = int(float(votes.split('K')[0])*1_000)
        else:
            votes = int(votes)
    except:
        votes = None
        print("Couldn't scrape the vote data")

    try:
        directors = []
        for i in html.find('div', {"data-testid":"title-pc-wide-screen"}).findAll('li', {"data-testid":"title-pc-principal-credit"})[0].findAll('a'):
            directors.append(i.text)
    except:
        directors = None
        print("Couldn't scrape the directors data")

    try:
        writers = []
        for i in html.find('div', {"data-testid":"title-pc-wide-screen"}).findAll('li', {"data-testid":"title-pc-principal-credit"})[1].findAll('a'):
            writers.append(i.text)
    except:
        writers = None
        print("Couldn't scrape the writers data")

    try:
        stars = []
        for i in html.find('div', {"data-testid":"title-pc-wide-screen"}).findAll('li', {"data-testid":"title-pc-principal-credit"})[2].find('ul', {"role":"presentation"}).findAll('a'):
            stars.append(i.text)
    except:
        stars = None
        print("Couldn't scrape the stars data")

    try:
        genres = []
        for i in html.find('li', {"data-testid":"storyline-genres"}).findAll('li'):
            genres.append(i.text)
    except:
        genres = None
        print("Couldn't scrape the genres data")

    row.append([A, B, C, D
                E, F, G])
    
    print(count, ": ", title, "...Done✔", sep="")
    print("Sleeping 5 seconds...")
    time.sleep(5)

In [ ]:
column = ["A", "B", "C", "D",
          "E", "F", "G"]

df = pd.DataFrame.from_records(row, columns=column)

In [ ]:
df.to_csv("data/sıcak_data.csv")
df.info()